In [26]:
import pandas as pd
import numpy as np
import sklearn
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
%matplotlib inline
import warnings
warnings.filterwarnings('ignore')

In [2]:
df=pd.read_csv('/Users/reema/Downloads/links.csv')
df


,web-scraper-order,web-scraper-start-url,links,links-href,court,case_number,case_date,appellate_court,appellate_id,appellate_date,region,city,judgment_text,appeal,pages,pages-href
0,1640298117-87,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38991,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,2,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
1,1640297824-62,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38903,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,5,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
2,1640298107-86,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٦٠٧ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38992,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٦٠٧,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٩٠,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,2,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
3,1640297908-69,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٣٣١ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39268,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٣٣١,تاريخها: ١٨/١١/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٣٧,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,4,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
4,1640297731-53,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٩٣٩٩ لعام ١٤٤١ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/38987,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٩٣٩٩,تاريخها: ١٥/١٠/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٣٤٣,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,5,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
97,1640297594-42,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٦٩٥٨ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39148,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٦٩٥٨,تاريخها: ٢٥/١٢/١٤٤٢,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٥٤٦,تاريخه: ٦/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,7,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
98,1640297971-74,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,القضية رقم ٤٦٤٦ لعام ١٤٤٢ هـ,https://sjp.moj.gov.sa/Filter/AhkamDetails/39236,المحكمة: المحكمة التجارية,رقم القضية - القرار: ٤٦٤٦,تاريخها: ٩/١/١٤٤٣,محكمة الاستئناف: المحكمة التجارية,رقم القرار: ٧٨٥,تاريخه: ٧/٣/١٤٤٣,المدينة: منطقة الرياض,المدينه: الرياض,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,الحمد لله والصلاة والسلام على رسول ﷲ أما بعد:\...,3,https://sjp.moj.gov.sa/Filter/FilterDataPager?...
99,1640298190-94,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,قرار الدائرة الخامسة بالمحكمة العليا رقم ٤٣١٥٠...,https://sjp.moj.gov.sa/Filter/AhkamDetails/39732,المحكمة: المحكمة العليا - الهيئة الدائمة,رقم القضية - القرار: ٤٣١٥٠١,تاريخها: ٨/٣/١٤٤٣,NaN,NaN,NaN,NaN,المدينه: الرياض,الحمد لله وحده والصلاة والسلام على نبينا محمد ...,NaN,NaN,NaN
100,1640298231-98,https://sjp.moj.gov.sa/Filter?isFilterButtonCl...,قرار الدائرة الخامسة بالمحكمة 

In [29]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 102 entries, 0 to 101
Data columns (total 16 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   web-scraper-order      102 non-null    object
 1   web-scraper-start-url  102 non-null    object
 2   links                  102 non-null    object
 3   links-href             102 non-null    object
 4   court                  102 non-null    object
 5   case_number            102 non-null    object
 6   case_date              102 non-null    object
 7   appellate_court        89 non-null     object
 8   appellate_id           89 non-null     object
 9   appellate_date         89 non-null     object
 10  region                 89 non-null     object
 11  city                   102 non-null    object
 12  judgment_text          102 non-null    object
 13  appeal                 89 non-null     object
 14  pages                  92 non-null     object
 15  pages-href             

In [4]:
df.duplicated().sum()

0

In [43]:
df_Train, df_test = train_test_split(df, test_size=0.2, random_state= 30)

df_train, df_val = train_test_split(df_Train, test_size=0.2, random_state= 30)

In [44]:
print(f'Shape of train:', df_train.shape)
print(f'Shape of validation:', df_val.shape)
print(f'Shape of test:', df_test.shape)

Shape of train: (64, 16)
Shape of validation: (17, 16)
Shape of test: (21, 16)


In [45]:
df_train.isnull().sum()

web-scraper-order        0
web-scraper-start-url    0
links                    0
links-href               0
court                    0
case_number              0
case_date                0
appellate_court          6
appellate_id             6
appellate_date           6
region                   6
city                     0
judgment_text            0
appeal                   6
pages                    7
pages-href               7
dtype: int64

In [46]:
df_val.isnull().sum()

web-scraper-order        0
web-scraper-start-url    0
links                    0
links-href               0
court                    0
case_number              0
case_date                0
appellate_court          4
appellate_id             4
appellate_date           4
region                   4
city                     0
judgment_text            0
appeal                   4
pages                    3
pages-href               3
dtype: int64

In [47]:
df_train.appellate_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    57
 محكمة الاستئناف: محكمة الاستئناف      1
 Name: appellate_court, dtype: int64,)

In [14]:
df_val.appellate_court.value_counts(),

(محكمة الاستئناف: المحكمة التجارية    13
 Name: appellate_court, dtype: int64,)

In [22]:
df_train['appellate_court'].fillna(df_train['appellate_court'].mode()[0], inplace = True)

In [21]:
df_val['appellate_court'].fillna(df_val['appellate_court'].mode()[0], inplace = True)

In [53]:
df_train.appellate_id.value_counts()

٧٥٧     2
١٩٦٩    2
٥٣٤٣    2
٥٤٩١    2
٦٥٤     2
٥٤٠٤    2
٥٥٨١    2
٧٩٠     2
٧٩٥     2
٥٣٨١    2
٢١٢٨    2
٦٣٩     2
٤٤٣٦    2
٩٦      2
٥١٧     2
٥٤٦     2
٤٤٢١    1
٧٨٥     1
٦٣٧     1
٨٤      1
٧٦      1
٢٢٢٩    1
٥٢٠٥    1
٣١٠٢    1
٧٠٤     1
٦٨٦     1
٤٣٧     1
٦٢٢     1
٥٥٢٨    1
٥٨٢     1
٦٤٦     1
٧٠٩     1
٣٤٧     1
٢٤٧     1
٣٢٤٧    1
٣٤٥٤    1
٣٥٩     1
٤٥٦٧    1
٦٢٣     1
٥٦٣٦    1
١٦٨٧    1
٥٤٢     1
Name: appellate_id, dtype: int64

In [56]:
df_val.appellate_id.value_counts()

٣٧      1
٦٢٣     1
٢٤٥     1
٥٤٢     1
٦٤٦     1
٦٨٦     1
٢٩٨٥    1
٢٦٣٠    1
٣٢٣     1
٥٨٢     1
٥٢٠٥    1
٦٣٧     1
٣٥٩     1
Name: appellate_id, dtype: int64

In [49]:
df_train.dropna(subset=['appellate_id'], inplace=True);

In [50]:
df_val.dropna(subset=['appellate_id'], inplace=True);

In [ ]:
#df_train['appellate_id'] = df_train['appellate_id'].astype(int)
#df['result'] = df['result'].str.replace(r'\D', '')

In [54]:
df_train['appellate_id'] = df_train['appellate_id'].str.replace('رقم القرار','')

In [55]:
df_val['appellate_id'] = df_val['appellate_id'].str.replace('رقم القرار','')

In [ ]:

df_train['appellate_court'] = df_train['appellate_court'].str.replace('رقم القرار','')